# Washington Crime Lab (RDDs and DataFrames)

In this lab, we'll explore some of the RDD concepts we've discussed. We'll be using a data set consisting of reported crimes in Washington D.C. from October 3, 2015 through October 2, 2016. This data comes from the [District of Columbia's Open Data Catalog](http://data.octo.dc.gov/). We'll use this data to explore some RDD transitions and actions.

## Exercises and Solutions

This notebook contains a number of exercises. Use the API docs for methods
<a href="http://spark.apache.org/docs/1.6.1/api/scala/index.html#org.apache.spark.rdd.RDD" target="_blank">common to all RDDs</a>,
plus the extra methods for
<a href="http://spark.apache.org/docs/1.6.1/api/scala/index.html#org.apache.spark.rdd.PairRDDFunctions" target="_blank">pair RDDs</a>, to look up transformations and actions. If, at any point, you're struggling with the solution to an exercise, feel free to look in the **Solutions** notebook (in the same folder as this lab).

## Let's get started.

## Load the data

The first step is to load the data. Run the following cell to create an RDD containing the data.

In [1]:
base_rdd = sc.textFile("/data/training/washington_crime_incidents_2013.csv")

NameError: name 'sc' is not defined

**Question**: Does the RDD _actually_ contain the data right now?

## Explore the data

Let's take a look at some of the data.

In [ ]:
for line in base_rdd.take(10):
  print(line)

Okay, there's a header. We'll need to remove that. But, since the file will be split into partitions, we can't just drop the first item. Let's figure out another way to do it.

In [ ]:
no_header_rdd = base_rdd.filter(lambda line: "REPORTDATETIME" not in line)

In [ ]:
for line in no_header_rdd.take(10):
  print(line)

### Exercise 1

Let's make things a little easier to handle, by converting the `noHeaderRDD` to an RDD containing Scala objects.

**TO DO**

* Split each line into its individual cells.
* Map the RDD into another RDD of appropriate `CrimeData` objects.

In [ ]:
# TODO
# Replace the <FILL-IN> sections with appropriate code.

# TAKE NOTE: We are deliberately only keeping the first five fields of
# each line, since that's all we're using in this lab. There's no sense
# in dragging around more data than we need.
from collections import namedtuple
from pprint import pprint

CrimeData = namedtuple('CrimeData', ['ccn', 'report_time', 'shift', 'offense', 'method'])

def map_line(line):
  columns = <FILL-IN>
  return <FILL-IN>

data_rdd = no_header_rdd.map(map_line)
pprint(data_rdd.take(10))

### Exercise 2

Next, group the data by type of crime (the "OFFENSE" column).

In [ ]:
# TODO
# Replace <FILL-IN> with the appropriate code.

grouped_by_offense_rdd = data_rdd.groupBy(lambda data: <FILL-IN>)
pprint(grouped_by_offense_rdd.take(10))

### Exercise 3
Next, create an RDD that counts the number of each offense. How many murders were there in the period covered by the data? How many assaults with a dangerous weapon?

In [ ]:
# TODO
# Replace <FILL-IN> with the appropriate code.

offense_counts = <FILL-IN>
for offense, count in <FILL-IN>:
  print("{0:30s} {1:d}".format(offense, count))

### Question

Run the following cell. Can you explain what happened? Is `collectAsMap()` a _transformation_ or an _action_?

In [ ]:
grouped_by_offense_rdd.collectAsMap()

### Exercise 4

How many partitions does the base RDD have? What about the `groupedByOffenseRDD` RDD? How can you find out?

**Hint**: Check the [API documentation](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD).

In [ ]:
# TODO
# Replace <FILL-IN> with the appropriate code.
print(base_rdd.<FILL-IN>)
print(grouped_by_offense_rdd.<FILL-IN>)

### Exercise 5

Since we're continually playing around with this data, we might as well cache it, to speed things up.

**Question**: Which RDD should you cache?

1. `baseRDD`
2. `noHeaderRDD`
3. `dataRDD`
4. None of them, because they're all still too big.
5. It doesn't really matter.

In [ ]:
# TODO
# Replace <FILL-IN> with the appropriate code.
<FILL-IN>.cache().count() # Why am I calling count() here?

### Exercise 6

Display the number of homicides by weapon (method).

In [ ]:
# TODO
# Replace <FILL-IN> with the appropriate code.
result_rdd1 = data_rdd.<FILL-IN>
print(result_rdd1.collect())

# BONUS: Make the output look better, using a for loop or a list comprehension.

### Exercise 7

During which police shift did the most crimes occur in the period covered by the data?

In [ ]:
# TODO
# Replace <FILL-IN> with the appropriate code.

# Hint: Start with the data_rdd
print(data_rdd.<FILL-IN>)

## Let's Switch to DataFrames

### Demonstration

Let's plot murders by month. DataFrames are useful for this one.

To do this properly, we'll need to parse the dates. That will require knowing their format. A quick sampling of the data will help.

In [ ]:
data_rdd.map(lambda item: item.report_time).take(30)

Okay. We can now parse the strings into actual `Date` objects.

**NOTE:** The DataFrame API does _not_ support schemas with `Date` objects in them. We'll need to convert the resulting `Date` to a `java.sql.Timestamp`.

Now, we can create the DataFrame. We'll start with the `dataRDD`, since it's already cached. Note that we're using the built-in `unix_timestamp` function, which parses
a string according to a Java [SimpleDateFormat](https://docs.oracle.com/javase/8/docs/api/java/text/SimpleDateFormat.html) pattern, returning the number of seconds since
the epoch (January 1, 1970 at midnight UTC), which we can then cast to a SQL timestamp type.

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
df0 = data_rdd.toDF()
df = df0.select(unix_timestamp(df0['report_time'], 'M/dd/yyyy KK:mm:SS a').cast('timestamp').alias('report_time'),
                df0['shift'],
                df0['offense'],
                df0['method'])

In [ ]:
df.printSchema()

In [ ]:
df.show()

Let's use the built-in
<a href="http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$@month(e:org.apache.spark.sql.Column):org.apache.spark.sql.Column" target="_blank"><tt>month</tt></a>
function to extract the month. Note that the months are 0-indexed. Let's make the 1-indexed, since that makes more sense to us humans.

In [ ]:
df_with_month = df.select(df['*'], month(df['report_time']).alias('month'))

In [ ]:
(
  df_with_month.filter(df_with_month['offense'] == 'HOMICIDE')
               .select(df_with_month['month'], df_with_month['offense'])
               .groupBy('month')
               .count()
               .show()
)

What about all crimes per month?

In [ ]:
df_with_month.select(df_with_month['month']).groupBy('month').count().show()

### Exercise 8

Plot the frequency of crimes by hour of day.

In [ ]:
# TODO
# Replace <FILL-IN> with your code.

  <FILL-IN>.show()